# MuData quickstart

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/scverse/mudata/blob/master/docs/source/notebooks/quickstart_mudata.ipynb)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/scverse/mudata/master?labpath=docs%2Fsource%2Fnotebooks%2Fquickstart_mudata.ipynb)

Introducing multimodal data — `MuData` — objects built on top of [AnnData](https://anndata.readthedocs.io/en/latest/index.html), `mudata` library naturally enriches the Python ecosystem for data analysis to enable multimodal data analysis. Be sure to check tools that take advantage of this data format such as [muon](https://muon.readthedocs.io/) — the Python framework for multimodal omics analysis.

This notebooks provides an introduction to multimodal data objects.

In [1]:
%pip install mudata

In [3]:
import mudata as md
from mudata import MuData

## Multimodal objects

To see how multimodal objects behave, we will simulate some data first:

In [4]:
import numpy as np
np.random.seed(1)

n, d, k = 1000, 100, 10

z = np.random.normal(loc=np.arange(k), scale=np.arange(k)*2, size=(n,k))
w = np.random.normal(size=(d,k))
y = np.dot(z, w.T)
y.shape

(1000, 100)

Creating an `AnnData` object from the matrix will allow us to add annotations to its different dimensions (_"observations"_, e.g. samples, and measured _"variables"_):

In [5]:
from anndata import AnnData

adata = AnnData(y)
adata.obs_names = [f"obs_{i+1}" for i in range(n)]
adata.var_names = [f"var_{j+1}" for j in range(d)]
adata

AnnData object with n_obs × n_vars = 1000 × 100

We will go ahead and create a second object with data for the _same observations_ but for _different variables_:

In [6]:
d2 = 50
w2 = np.random.normal(size=(d2,k))
y2 = np.dot(z, w2.T)

adata2 = AnnData(y2)
adata2.obs_names = [f"obs_{i+1}" for i in range(n)]
adata2.var_names = [f"var2_{j+1}" for j in range(d2)]
adata2

AnnData object with n_obs × n_vars = 1000 × 50

We can now wrap these two objects into a `MuData` object:

In [7]:
mdata = MuData({"A": adata, "B": adata2})
mdata

MuData object with n_obs × n_vars = 1000 × 150
  2 modalities
    A:	1000 x 100
    B:	1000 x 50

_Observations_ and _variables_ of the `MuData` object are global, which means that observations with the identical name (`.obs_names`) in different modalities are considered to be the same observation. This also means variable names (`.var_names`) should be unique.

This is reflected in the object description above: `mdata` has 1000 _observations_ and 150=100+50 _variables_.

### Variable mappings

Upon construction of a `MuData` object, a global binary mapping between _observations_ and individual modalities is created as well as between _variables_ and modalities.

Since all the observations are the same across modalities in `mdata`, all the values in the _observations_ mappings are set to `True`:

In [8]:
np.sum(mdata.obsm["A"]) == np.sum(mdata.obsm["B"]) == n

True

For variables, those are 150-long vectors, e.g. for the `A` modality — with 100 `True` values followed by 50 `False` values:

In [9]:
mdata.varm['A']

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

### Object references

Importantly, individual modalities are stored as references to the original objects.

In [10]:
# Add some unstructured data to the original object
adata.uns["misc"] = {"adata": True}

In [11]:
# Access modality A via the .mod attribute
mdata.mod["A"].uns["misc"]

{'adata': True}

This is also why the `MuData` object has to be updated in order to reflect the latest changes to the modalities it includes:

In [12]:
adata2.var_names = ["var_ad2_" + e.split("_")[1] for e in adata2.var_names]

In [13]:
print(f"Outdated variables names: ...,", ", ".join(mdata.var_names[-3:]))
mdata.update()
print(f"Updated variables names: ...,", ", ".join(mdata.var_names[-3:]))

Outdated variables names: ..., var2_48, var2_49, var2_50
Updated variables names: ..., var_ad2_48, var_ad2_49, var_ad2_50


### Common observations

While `mdata` is comprised of the same observations for both modalities, it is not always the case in the real world where some data might be missing. By design, `mudata` accounts for these scenarios since there's no guarantee observations are the same — or even intersecting — for a `MuData` instance.

It's worth noting that other tools might provide convenience functions for some common scenarios of dealing with missing data, such as `intersect_obs()` [implemented in muon](https://muon.readthedocs.io/en/latest/api/generated/muon.pp.intersect_obs.html?highlight=intersect_obs).

### Rich representation

Some notebook environments such as Jupyter/IPython allow for the [rich object representation](https://ipython.readthedocs.io/en/stable/config/integrating.html). This is what `mudata` uses in order to provide an optional HTML representation that allows to interactively explore `MuData` objects. While the dataset in our example is not the most comprehensive one, here is how it looks like:

In [14]:
with md.set_options(display_style = "html", display_html_expand = 0b000):
    display(mdata)

A,bool,numpy.ndarray,
B,bool,numpy.ndarray,
misc,dict,1 element,adata: True


Running `md.set_options(display_style = "html")` will change the setting for the current Python session. 

The flag `display_html_expand` has three bits that correspond to

1. `MuData` attributes, 
1. modalities, 
1. `AnnData` attributes, 

and indicates if the fields should be expanded by default (`1`) or collapsed under the `<summary>` tag (`0`).

### .h5mu files

`MuData` objects were designed to be serialized into `.h5mu` files. Modalities are stored under their respective names in the `/mod` HDF5 group of the `.h5mu` file. Each individual modality, e.g. `/mod/A`, is stored in the same way as it would be stored in the `.h5ad` file.

In [15]:
import tempfile

# Create a temporary file
temp_file = tempfile.NamedTemporaryFile(mode="w", suffix=".h5mu", prefix="muon_getting_started_")

mdata.write(temp_file.name)
mdata_r = md.read(temp_file.name, backed=True)
mdata_r

MuData object with n_obs × n_vars = 1000 × 150 backed at '/var/folders/6f/ht39xtd54tv4n5p35ccnpgf80000gn/T/muon_getting_started_63ro6syq.h5mu'
  2 modalities
    A:	1000 x 100
      uns:	'misc'
    B:	1000 x 50

Individual modalities are backed as well — inside the `.h5mu` file:

In [16]:
mdata_r["A"].isbacked

True

The rich representation would also reflect the _backed_ state of `MuData` objects when they are loaded from `.h5mu` files in the read-only mode and would point to the respective file:

In [17]:
with md.set_options(display_style = "html", display_html_expand = 0b000):
    display(mdata_r)

A,bool,numpy.ndarray,
B,bool,numpy.ndarray,
misc,dict,1 element,adata: True


## Multimodal methods

When the `MuData` object is prepared, it is up to multimodal methods to be used to make sense of the data. The most simple and naïve approach is to concatenate matrices from multiple modalities to perform e.g. dimensionality reduction.

In [18]:
x = np.hstack([mdata.mod["A"].X, mdata.mod["B"].X])
x.shape

(1000, 150)

We can write a simple function to run principal component analysis on such a concatenated matrix. `MuData` object provides a place to store multimodal embeddings — `MuData.obsm`. It is similar to how the embeddings generated on invidual modalities are stored, only this time it is saved inside the `MuData` object rather than in `AnnData.obsm`.

In [19]:
def simple_pca(mdata):
    from sklearn import decomposition
    
    x = np.hstack([m.X for m in mdata.mod.values()])

    pca = decomposition.PCA(n_components=2)
    components = pca.fit_transform(x)

    # By default, methods operate in-place
    # and embeddings are stored in the .obsm slot
    mdata.obsm["X_pca"] = components
    
    return 

In [20]:
simple_pca(mdata)
print(mdata)

MuData object with n_obs × n_vars = 1000 × 150
  obsm:	'X_pca'
  2 modalities
    A:	1000 x 100
      uns:	'misc'
    B:	1000 x 50


In reality, however, having different modalities often means that the features between them come from different generative processes and are not comparable.

This is where special multimodal integration methods come into play. For omics technologies, these methods are frequently addressed as _multi-omics integration methods_. `MuData` objects make it easy for the new methods to be easily applied to such data, and some of them [are implemented in muon](https://muon.readthedocs.io/en/latest/omics/multi.html).
